In [ ]:
!pip install geopy
!pip install tzwhere
!pip install skyfield
!pip install timezonefinder

In [ ]:
from datetime import datetime
from geopy import Nominatim
from tzwhere import tzwhere
from pytz import timezone, utc

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.patches import Circle
from timezonefinder import TimezoneFinder

from skyfield.api import Star, load, wgs84
from skyfield.data import hipparcos
from skyfield.projections import build_stereographic_projection

In [ ]:
# load celestial data

# de421 shows position of earth and sun in space
eph = load('de421.bsp')

# hipparcos dataset contains star location data
with load.open(hipparcos.URL) as f:
    stars = hipparcos.load_dataframe(f)

In [ ]:
location = 'Venice, Italy'
when = '2025-03-25 12:00'

In [ ]:
# get latitude and longitude of our location 
locator = Nominatim(user_agent='myGeocoder')
location = locator.geocode(location)
lat, long = location.latitude, location.longitude

In [ ]:
# convert date string into datetime object
dt = datetime.strptime(when, '%Y-%m-%d %H:%M')

# Define datetime and convert to UTC based on our timezone
tf = TimezoneFinder()
timezone_str = tf.timezone_at(lng=long, lat=lat)
local = timezone(timezone_str)

# get UTC from local timezone and datetime
local_dt = local.localize(dt, is_dst=None)
utc_dt = local_dt.astimezone(utc)

In [ ]:
print(f"Date string: {when}")
print(f"Latitude: {lat}, Longitude: {long}")
print(f"Timezone: {timezone_str}")
print(f"Localized datetime: {local_dt}")
print(f"UTC datetime: {utc_dt}")

In [ ]:
# find location of earth and sun and set the observer position
sun = eph['sun']
earth = eph['earth']

# define observation time from our UTC datetime
ts = load.timescale()
t = ts.from_datetime(utc_dt)

# define an observer using the world geodetic system data
observer = wgs84.latlon(latitude_degrees=lat, longitude_degrees=long).at(t)

# define the position in the sky where we will be looking
position = observer.from_altaz(alt_degrees=90, az_degrees=0)

In [ ]:
# center the observation point in the middle of the sky
ra, dec, distance = observer.radec()
center_object = Star(ra=ra, dec=dec)

# find where our center object is relative to earth and build a projection with 180 degree view
center = earth.at(t).observe(center_object)
projection = build_stereographic_projection(center)
field_of_view_degrees = 180.0

# calculate star positions and project them onto a plain space
star_positions = earth.at(t).observe(Star.from_dataframe(stars))
stars['x'], stars['y'] = projection(star_positions)

Let's assume that the size of our star represents it's relative magnitude.  

Given $F_x$ - the observed irradiance using spectral filter x, and $F_{x,0}$ is the reference flux (zero-point) for that photometric filter, magnitude $m_x$ is 


$$ m_x = -2.5 \cdot log_{10} \big( \frac{F_x}{F_{x,0}} \big)  $$

Let the size of our biggest star in our chart which represents the brightest star be $S_0$ and the size of any star $i$ be represented by $S_i$ and its apparent magnitude be represented by $m_i$. Then,

$$ m_i = -2.5 \cdot log_{10} \big( \frac{S_i}{S_0} \big)  $$

Solving for $S_i$,

$$ S_i = S_0 \cdot 10 ^{m_i/-2.5} $$

In [ ]:
chart_size = 10
max_star_size = 100
limiting_magnitude = 10

bright_stars = (stars.magnitude <= limiting_magnitude)
magnitude = stars['magnitude'][bright_stars]

In [ ]:
fig, ax = plt.subplots(figsize=(chart_size, chart_size))
    
border = plt.Circle((0, 0), 1, color='navy', fill=True)
ax.add_patch(border)

marker_size = max_star_size * 10 ** (magnitude / -2.5)

ax.scatter(stars['x'][bright_stars], stars['y'][bright_stars],
           s=marker_size, color='white', marker='.', linewidths=0, 
           zorder=2)

horizon = Circle((0, 0), radius=1, transform=ax.transData)
for col in ax.collections:
    col.set_clip_path(horizon)


# other settings
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
plt.axis('off')

plt.show()